# Data Cleaning

In [114]:
# Install unidecode in jupyter kernel
import sys
!{sys.executable} -m pip install unidecode

In [115]:
# import modules
import numpy as np
import pandas as pd
import unidecode

In [116]:
# define some useful helper functions

# resuable function to print dimensions in a more descriptive manner
def print_shape(df):
    print("Data dimensions: ", df.shape[0], "rows and", df.shape[1], "columns")
    
# function that checks for alphabet match between two strings based on threshold value
def str_alphabet_match(str1, str2):
    clean_str1 = unidecode.unidecode(''.join(char for char in str1 if char.isalpha())).lower()
    clean_str2 = unidecode.unidecode(''.join(char for char in str2 if char.isalpha())).lower()
    return sorted(clean_str1) == sorted(clean_str2)

# lewenshtien distance for fuzzy string matching
def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return "The strings are {} edits away".format(distance[row][col])

In [117]:
# import main data file
raw_data = pd.read_csv("./data/country_profile_variables.csv")
raw_data.head()

,country,Region,Surface area (km2),Population in thousands (2017),"Population density (per km2, 2017)","Sex ratio (m per 100 f, 2017)",GDP: Gross domestic product (million current US$),"GDP growth rate (annual %, const. 2005 prices)",GDP per capita (current US$),Economy: Agriculture (% of GVA),...,Mobile-cellular subscriptions (per 100 inhabitants).1,Individuals using the Internet (per 100 inhabitants),Threatened species (number),Forested area (% of land area),CO2 emission estimates (million tons/tons per capita),"Energy production, primary (Petajoules)",Energy supply per capita (Gigajoules),"Pop. using improved drinking water (urban/rural, %)","Pop. using improved sanitation facilities (urban/rural, %)",Net Official Development Assist. received (% of GNI)
0,Afghanistan,SouthernAsia,652864,35530,54.4,106.3,20270,-2.4,623.2,23.3,...,8.3,42,2.1,9.8/0.3,63,5,78.2/47.0,45.1/27.0,21.43,-99
1,Albania,SouthernEurope,28748,2930,106.9,101.9,11541,2.6,3984.2,22.4,...,63.3,130,28.2,5.7/2.0,84,36,94.9/95.2,95.5/90.2,2.96,-99
2,Algeria,NorthernAfrica,2381741,41318,17.3,102.0,164779,3.8,4154.1,12.2,...,38.2,135,0.8,145.4/3.7,5900,55,84.3/81.8,89.8/82.2,0.05,-99
3,American Samoa,Polynesia,199,56,278.2,103.6,-99,-99,-99.0,-99,...,-99,92,87.9,-99,-99,-99,100.0/100.0,62.5/62.5,-99,-99
4,Andorra,SouthernEurope,468,77,163.8,102.3,2812,0.8,39896.4,0.5,...,96.9,13,34.0,0.5/6.4,1,119,100.0/100.0,100.0/100.0,-99,-99


In [118]:
# check data shape, columns, column types, etc
# print raw data dimens
print_shape(raw_data)

Data dimensions:  229 rows and 50 columns


***Some inital observations***

1. first interesting point noticed, world has 193 sovereign states and 249 ISO code territories but the data set has 229 rows which is in between the 2 values. We need to explore why this is the case. This will be done in the section titled ***DATA CLEANING - Countries VS territories***


2. It appears that the value "-99" appears quite frequently for the last column "Net Official Development Assist. received (% of GNI)". It is possible that the whole column has the same value. This will be further explored in ***DATA CLEANING - Dropping columns***


3. Some columns such as "Pop. using improved drinking water (urban/rural, %)" are in the format of "number/number". This object type is difficult to deal with. These columns need to be split into 2. This wille be done in the section titled ***DATA CLEANING - Handling data types***

# DATA CLEANING - Countries VS territories

In [119]:
# first interesting point noticed, world has 193 sovereign states and 249 ISO code territories 
# but the data set has 229 rows which is in between the 2 values
# Let's explore why this is the case

# import external data for countries and ISO codes
territories = pd.read_csv("./data/world.csv")
countries = pd.read_csv( "./data/countries.csv")

print("Territories \n", territories.head())
print_shape(territories)
print("\n\n\n")
print("Countries \n", countries.head())
print_shape(countries)

Territories 
     id alpha2 alpha3            name
0    4     af    afg     Afghanistan
1  248     ax    ala   Åland Islands
2    8     al    alb         Albania
3   12     dz    dza         Algeria
4   16     as    asm  American Samoa
Data dimensions:  249 rows and 4 columns




Countries 
    id alpha2 alpha3         name
0   4     af    afg  Afghanistan
1   8     al    alb      Albania
2  12     dz    dza      Algeria
3  20     ad    and      Andorra
4  24     ao    ago       Angola
Data dimensions:  193 rows and 4 columns


In [120]:
# compare country names in raw_data df with names in countries df

# convert df columns into series objects
s1 = raw_data["country"].squeeze()
s2 = countries["name"].squeeze()

# iterate over raw_data and find names that are not in countries list
count = 0
print("names that appear in raw_data but not in countries list\n")
names_to_drop = []
for s1Index, s1Value in s1.items():
    nameFound = False
    for s2Index, s2Value in s2.items():
#         if s1Value.lower().rstrip() == s2Value.lower().rstrip():
#             nameFound = True
          if str_alphabet_match(s1Value, s2Value):
            nameFound = True
    if (nameFound == False):
        count += 1
        print(s1Value)
        names_to_drop.append(s1Value)
print("Total:", count)
print()

names that appear in raw_data but not in countries list

American Samoa
Anguilla
Aruba
Bermuda
Bonaire, Sint Eustatius and Saba
British Virgin Islands
Cayman Islands
Channel Islands
China, Hong Kong SAR
China, Macao SAR
Cook Islands
Falkland Islands (Malvinas)
Faroe Islands
French Guiana
French Polynesia
Gibraltar
Greenland
Guadeloupe
Guam
Holy See
Isle of Man
Martinique
Mayotte
Montserrat
New Caledonia
Niue
Northern Mariana Islands
Puerto Rico
Saint Helena
Saint Pierre and Miquelon
Sint Maarten (Dutch part)
State of Palestine
Swaziland
The former Yugoslav Republic of Macedonia
Tokelau
Turks and Caicos Islands
United Kingdom
United States Virgin Islands
Wallis and Futuna Islands
Western Sahara
Total: 40



***Findings from comparing countries with country list***

At this point we will drop territories from our raw data and only keep the countries

Swaziland is surfaced as it is still called by its old name Eswatini in the country list
United Kingdom which is also a country is surfaced due to a naming descrepency
Macedonia was also surfaced due to naming descrepency

The rest of the names surfaced can be dropped as they are territories :)

In [121]:
# exclude the above 3 countries from names to drop
names_to_drop.remove("Swaziland")
names_to_drop.remove("The former Yugoslav Republic of Macedonia")
names_to_drop.remove("United Kingdom")
raw_data = raw_data.drop(raw_data[raw_data.country.isin(names_to_drop)].index)

# DATA CLEANING - Dropping columns

In [122]:
# do statistical description to verify if all the values in last column are the same
print_shape(raw_data)
raw_data.describe()

Data dimensions:  192 rows and 50 columns


,Population in thousands (2017),"Population density (per km2, 2017)","Sex ratio (m per 100 f, 2017)",GDP: Gross domestic product (million current US$),GDP per capita (current US$),Economy: Industry (% of GVA),Economy: Services and other activity (% of GVA),Agricultural production index (2004-2006=100),Food production index (2004-2006=100),Urban population (% of total population),Health: Total expenditure (% of GDP),Seats held by women in national parliaments %,Individuals using the Internet (per 100 inhabitants),CO2 emission estimates (million tons/tons per capita),"Energy production, primary (Petajoules)",Net Official Development Assist. received (% of GNI)
count,1.920000e+02,192.000000,192.000000,1.920000e+02,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000,192.0
mean,3.896384e+04,333.787500,101.906250,3.807681e+05,14121.805729,28.610417,59.281771,111.671875,112.536458,57.125000,4.573958,21.298437,223.911458,2946.734375,83.859375,-99.0
std,1.447859e+05,1962.384692,21.362081,1.608619e+06,23419.945548,12.767695,14.407060,45.227310,45.668263,23.546679,15.398910,11.825115,315.382702,10763.475415,123.471709,0.0
min,1.100000e+01,2.000000,84.900000,3.300000e+01,144.500000,4.000000,14.900000,-99.000000,-99.000000,8.400000,-99.000000,0.000000,1.000000,-99.000000,-99.000000,-99.0
25%,2.006250e+03,35.800000,96.675000,6.547750e+03,1718.150000,20.075000,50.450000,102.000000,102.000000,38.600000,4.775000,12.375000,61.750000,31.000000,19.000000,-99.0
50%,8.606000e+03,86.050000,99.000000,2.805350e+04,5071.950000,27.150000,60.000000,114.000000,116.000000,58.150000,6.250000,19.950000,110.500000,246.000000,44.000000,-99.0
75%,2.895175e+04,206.400000,101.225000,1.864740e+05,15556.550000,34.100000,70.100000,134.000000,134.000000,76.875000,8.325000,29.525000,239.750000,1261.000000,99.750000,-99.0
max,1.409517e+06,25969.800000,301.200000,1.803665e+07,169491.800000,79.900000,88.600000,199.000000,199.000000,100.000000,17.100000,61.300000,2358.000000,101394.000000,952.000000,-99.0


***Findings from statistical description table summary***

1. Noticed "-99" for many minimum values, it appears that -99 is used as a placeholder when data is not available. These columns need to be cleaned if we are going to include them in our final model train data


2. Confirmed that all values in last column "Net Official Development Assist. received (% of GNI)" are -99 since mean is -99 and standard deviation is zero. This column is hence not useful for categorisation and should be dropped.


In [123]:
# dropping last column
raw_data = raw_data.iloc[:,:-1]
raw_data.head()

,country,Region,Surface area (km2),Population in thousands (2017),"Population density (per km2, 2017)","Sex ratio (m per 100 f, 2017)",GDP: Gross domestic product (million current US$),"GDP growth rate (annual %, const. 2005 prices)",GDP per capita (current US$),Economy: Agriculture (% of GVA),...,Mobile-cellular subscriptions (per 100 inhabitants),Mobile-cellular subscriptions (per 100 inhabitants).1,Individuals using the Internet (per 100 inhabitants),Threatened species (number),Forested area (% of land area),CO2 emission estimates (million tons/tons per capita),"Energy production, primary (Petajoules)",Energy supply per capita (Gigajoules),"Pop. using improved drinking water (urban/rural, %)","Pop. using improved sanitation facilities (urban/rural, %)"
0,Afghanistan,SouthernAsia,652864,35530,54.4,106.3,20270,-2.4,623.2,23.3,...,61.6,8.3,42,2.1,9.8/0.3,63,5,78.2/47.0,45.1/27.0,21.43
1,Albania,SouthernEurope,28748,2930,106.9,101.9,11541,2.6,3984.2,22.4,...,106.4,63.3,130,28.2,5.7/2.0,84,36,94.9/95.2,95.5/90.2,2.96
2,Algeria,NorthernAfrica,2381741,41318,17.3,102.0,164779,3.8,4154.1,12.2,...,113.0,38.2,135,0.8,145.4/3.7,5900,55,84.3/81.8,89.8/82.2,0.05
4,Andorra,SouthernEurope,468,77,163.8,102.3,2812,0.8,39896.4,0.5,...,88.1,96.9,13,34.0,0.5/6.4,1,119,100.0/100.0,100.0/100.0,-99
5,Angola,MiddleAfrica,1246700,29784,23.9,96.2,117955,3.0,4714.1,6.8,...,60.8,12.4,146,46.5,34.8/1.4,3902,25,75.4/28.2,88.6/22.5,0.42


# DATA CLEANING - Handling data types & dirty values

In [124]:
# first lets check the column data types 
#to figure out which columns are objects and could potentially be converted to numbers
print(raw_data.dtypes)

country                                                        object
Region                                                         object
Surface area (km2)                                             object
Population in thousands (2017)                                  int64
Population density (per km2, 2017)                            float64
Sex ratio (m per 100 f, 2017)                                 float64
GDP: Gross domestic product (million current US$)               int64
GDP growth rate (annual %, const. 2005 prices)                 object
GDP per capita (current US$)                                  float64
Economy: Agriculture (% of GVA)                                object
Economy: Industry (% of GVA)                                  float64
Economy: Services and other activity (% of GVA)               float64
Employment: Agriculture (% of employed)                        object
Employment: Industry (% of employed)                           object
Employment: Services

In [125]:
# first will first split the columns with format "number/number" 
# female / male columns
# Labour force participation (female/male pop. %)
# Life expectancy at birth (females/males, years)
# Education: Primary gross enrol. ratio (f/m per 100 pop.)
# Education: Secondary gross enrol. ratio (f/m per 100 pop.)
# Education: Tertiary gross enrol. ratio (f/m per 100 pop.)
    
def do_splitting(df, colName, colALabel="a", colBLabel="b", delimiter="/"):
        ogCol = df[colName]
        colA = ogCol.apply(lambda x : x.split(delimiter)[0] if isinstance(x, str) else x)
        colB = ogCol.apply(lambda x : x.split(delimiter)[1] if isinstance(x, str) and "/" in x else -99)
        df[colName + "-" + colALabel] = colA
        df[colName + "-" + colBLabel] = colB

def split_cols(df, columns, colALabel="a", colBLabel="b", delimiter="/"):
    if isinstance(columns, str):
        do_splitting(df, columns, colALabel, colBLabel, delimiter)
    elif isinstance(columns, list):
        for colName in columns:
            do_splitting(df, colName, colALabel, colBLabel, delimiter)
    
    df.drop(columns=columns, inplace=True)
    

# female male cols
female_male_cols = ['Labour force participation (female/male pop. %)',
'Life expectancy at birth (females/males, years)',
'Education: Primary gross enrol. ratio (f/m per 100 pop.)',
'Education: Secondary gross enrol. ratio (f/m per 100 pop.)',
'Education: Tertiary gross enrol. ratio (f/m per 100 pop.)']

split_cols(raw_data, female_male_cols, "female", "male", "/")

# other cols
split_cols(raw_data, "Population age distribution (0-14 / 60+ years, %)", "0-14", "60+ years")
split_cols(raw_data, "International migrant stock (000/% of total pop.)", "in thousands", "as % of total pop")
split_cols(raw_data, "Forested area (% of land area)", "A", "B")
split_cols(raw_data, "Energy supply per capita (Gigajoules)", "A", "B")
split_cols(raw_data, "Pop. using improved drinking water (urban/rural, %)", "Urban", "Rural")


# removing dirty values
raw_data = raw_data.replace(to_replace = "...", value = -99)
raw_data = raw_data.replace(to_replace ="^~", value = 0, regex=True)
raw_data = raw_data.replace(to_replace="^-", value = 0, regex=True)
raw_data = raw_data.replace(to_replace = "...", value = -99)

# convert entire data frame to numeric
raw_data = raw_data.convert_dtypes()
raw_data.dtypes

temp_df = raw_data.drop(columns=["country", "Region"])
temp_df = temp_df.apply(pd.to_numeric)
temp_df["country"] = raw_data["country"]
temp_df["Region"] = raw_data["Region"]
raw_data = temp_df

In [126]:
raw_data.head()

,Surface area (km2),Population in thousands (2017),"Population density (per km2, 2017)","Sex ratio (m per 100 f, 2017)",GDP: Gross domestic product (million current US$),"GDP growth rate (annual %, const. 2005 prices)",GDP per capita (current US$),Economy: Agriculture (% of GVA),Economy: Industry (% of GVA),Economy: Services and other activity (% of GVA),...,International migrant stock (000/% of total pop.)-in thousands,International migrant stock (000/% of total pop.)-as % of total pop,Forested area (% of land area)-A,Forested area (% of land area)-B,Energy supply per capita (Gigajoules)-A,Energy supply per capita (Gigajoules)-B,"Pop. using improved drinking water (urban/rural, %)-Urban","Pop. using improved drinking water (urban/rural, %)-Rural",country,Region
0,652864,35530,54.4,106.3,20270,0.0,623.2,23.3,23.3,53.3,...,382.4,1.2,9.8,0.3,78.2,47.0,45.1,27.0,Afghanistan,SouthernAsia
1,28748,2930,106.9,101.9,11541,2.6,3984.2,22.4,26.0,51.7,...,57.6,2.0,5.7,2.0,94.9,95.2,95.5,90.2,Albania,SouthernEurope
2,2381741,41318,17.3,102.0,164779,3.8,4154.1,12.2,37.3,50.5,...,242.4,0.6,145.4,3.7,84.3,81.8,89.8,82.2,Algeria,NorthernAfrica
4,468,77,163.8,102.3,2812,0.8,39896.4,0.5,10.8,88.6,...,42.1,59.7,0.5,6.4,100.0,100.0,100.0,100.0,Andorra,SouthernEurope
5,1246700,29784,23.9,96.2,117955,3.0,4714.1,6.8,51.2,42.0,...,106.8,0.4,34.8,1.4,75.4,28.2,88.6,22.5,Angola,MiddleAfrica


# Filling in missing values

In [127]:
# locate negative 99 values
print("Below this:")
for (col, data) in raw_data.iteritems():
    if col in ["country", "Region"]:
        continue
    df = raw_data.loc[raw_data[col] == -99]
    if not df.empty:
        print(df[["country", col]])
        print(df.shape)
        print("\n")

Below this:
                   country  Employment: Agriculture (% of employed)
7      Antigua and Barbuda                                    -99.0
59                Dominica                                    -99.0
108               Kiribati                                    -99.0
126       Marshall Islands                                    -99.0
154                  Palau                                    -99.0
170  Saint Kitts and Nevis                                    -99.0
175             San Marino                                    -99.0
(7, 59)


                   country  Employment: Industry (% of employed)
7      Antigua and Barbuda                                 -99.0
59                Dominica                                 -99.0
108               Kiribati                                 -99.0
126       Marshall Islands                                 -99.0
154                  Palau                                 -99.0
170  Saint Kitts and Nevis                  

In [128]:
# patching government expenditure on healthcare missing data from an external source
# external source: https://data.worldbank.org/indicator/SE.XPD.TOTL.GD.ZS?end=2017&start=2017

# load education expenditure data from secondary source
edu_exp = pd.read_csv('./data/government_expenditure_on_eduction_of_gdp.csv')

# drop data for all years except 2017
edu_exp = edu_exp[["Country Name", "2017"]]

# locate countries in raw_data with -99 for education expenditure
countries_with_missing_data = raw_data[raw_data["Education: Government expenditure (% of GDP)"] == -99]["country"]
count = 0

# find and replace data from edu_exp into raw_exp
for index1, country in countries_with_missing_data.iteritems():
    for index2, row in edu_exp.iterrows():
        # use levenshtein ratio for fuzzy string matching since country names in raw_data and edu_exp are not exactly the same
        if levenshtein_ratio_and_distance(country.lower(), row["Country Name"].lower(), True) >= 0.8:
            if (row["2017"]):
                raw_data.at[index1, "Education: Government expenditure (% of GDP)"] = row["2017"]
                countries_with_missing_data.pop(index1)
                count += 1
print(count, "/ 34 countries with missing expenditure on healthcare data found and corrected")

# log countries that were not detected by the fuzzy string matching
print("remaining ", 34 - count, "countries: ")
print(countries_with_missing_data)

28 / 34 countries with missing expenditure on healthcare data found and corrected
remaining  6 countries: 
14                                       Bahamas
48                                         Congo
62                                         Egypt
201    The former Yugoslav Republic of Macedonia
222           Venezuela (Bolivarian Republic of)
226                                        Yemen
Name: country, dtype: string


In [135]:
# manually replace data for remaining 6 countries
raw_data.at[14, "Education: Government expenditure (% of GDP)"] = edu_exp[edu_exp["Country Name"] == "Bahamas, The"]["2017"]
raw_data.at[48, "Education: Government expenditure (% of GDP)"] = edu_exp[edu_exp["Country Name"] == "Congo, Rep."]["2017"]
raw_data.at[62, "Education: Government expenditure (% of GDP)"] = edu_exp[edu_exp["Country Name"] == "Egypt, Arab Rep."]["2017"]
raw_data.at[201, "Education: Government expenditure (% of GDP)"] = edu_exp[edu_exp["Country Name"] == "North Macedonia"]["2017"]
raw_data.at[222, "Education: Government expenditure (% of GDP)"] = edu_exp[edu_exp["Country Name"] == "Venezuela, RB"]["2017"]
raw_data.at[226, "Education: Government expenditure (% of GDP)"] = edu_exp[edu_exp["Country Name"] == "Yemen, Rep."]["2017"]

# check if all have been replaced (empty series should be printed if successful)
countries_with_missing_data = raw_data[raw_data["Education: Government expenditure (% of GDP)"] == -99]["country"]
print(countries_with_missing_data)


Series([], Name: country, dtype: string)


In [136]:
clean_data = raw_data

%store clean_data

Stored 'clean_data' (DataFrame)
